# 2D flying-snake at Re=2000 and AoA=35deg (snappyHexMesh + OpenFOAM-2.3.0) with smallest grid-spacing h=0.004

---

This notebook heavily relies on the Python package `snake`, available on [GitHub](https://github.com/mesnardo/snake).
`snake` was written to post-process and compare the numerical solution from 4 different softwares ([`cuIBM`](https://github.com/barbagroup/cuIBM), [`PetIBM`](https://github.com/barbagroup/PetIBM), [`IBAMR`](https://github.com/IBAMR/IBAMR), and [`OpenFOAM`](http://www.openfoam.com/)).

In [ ]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.geometry import Geometry
from snake.openfoam import OBJFile
from snake.openfoam.simulation import OpenFOAMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [ ]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Last time we ran this notebook, the output of the cell above was:
```
Python version: 2.7.12 |Anaconda 2.5.0 (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1.2
```

---

### Description

We aim to compute the 2D flow around a snake cross-section with angle-of-attack 35 degrees at Reynolds number 2000 using IcoFOAM, the laminar incompressible solver of OpenFOAM.

The computational mesh, generated with the OpenFOAM mesh utility snappyHexMesh, contains about 3.4 million cells.

---

### General information

* software: `OpenFOAM-2.3.0`
* post-processing software: `ParaView-4.1` (included in `OpenFOAM-2.3.1`)
* submission date: 2015/10/30 (mesnardo)
* running date: 2015/10/30
* machine: [`Colonial One`](https://ots.columbian.gwu.edu/colonial-one-high-performance-computing-initiative) (short queue, 16 processes, 1 node)

In [ ]:
simulation = OpenFOAMSimulation(description='IcoFOAM')

---

### Computational domain and mesh

The mesh contains about 3.4 million cells and was created with the 3D OpenFOAM mesh utility, `snappyHexMesh`.

To generate the mesh, you need first to create the necessary .obj files (`snake.obj`, `boxNear.obj`, and `boxWake.obj`). To do that, run the cell below.
Afterwards, you can generate the mesh with the command-line:
    > runSnappyHexMesh.sh

In [ ]:
# create triSurface directory if not present
save_directory = os.path.join(simulation.directory,
                              'constant',
                              'triSurface')
if not os.path.isdir(save_directory):
    os.makedirs(save_directory)

# save the bluff-body as .obj file
body = Geometry(file_path=os.path.join(os.environ['SNAKE'],
                                       'resources',
                                       'geometries',
                                       'flyingSnake2d',
                                       'flyingSnake2dAoA35.dat'))
body.write('tmpBody.dat')
body = OBJFile.Body2d('tmpBody.dat',
                      name='snake',
                      extrusion_limits=(0.0, 1.0))
body.write(save_directory=save_directory)
os.remove('tmpBody.dat')

# create two boxes of refinement as .obj files
box = OBJFile.Box2d('boxNear',
                    bottom_left=(-1.0, -2.0),
                    top_right=(10.0, 2.0),
                    n=(800, 400))
box.write(save_directory)
box = OBJFile.Box2d('boxWake',
                    bottom_left=(-2.0, -4.0),
                    top_right=(15.0, 4.0),
                    n=(700, 300))
box.write(save_directory)

---

### Boundary conditions

In [ ]:
boundary_info = '0/U'
%cat $boundary_info
boundary_info = '0/p'
%cat $boundary_info

---

### Simulation parameters

In [ ]:
simulation_info = 'system/controlDict'
%cat $simulation_info
simulation_info = 'system/fvSolution'
%cat $simulation_info
simulation_info = 'system/fvScheme'
%cat $simulation_info

---

### Running the case

To run the the case in parallel , we need to decompose the domain:
    > decomposeParDict

To run the case the simulation:
    > mpirun -n <nprocs> icoFOAM -parallel > log.icoFoam

To reconstruct the parallel solution:
    > reconstructPar

To generate the vorticity field at each time saved:
    > vorticity

---